In [5]:
import numpy as np
import random
import json
from gym import Env
from gym.spaces import Discrete, Box
import copy

# ===============================
# Environment with Pokémon Switching
# ===============================
class EnhancedPokemonBattleEnv(Env):
    def __init__(self, pokemon_data_path="/content/pokemon_trial.json", opponent_q_table=None):
        super(EnhancedPokemonBattleEnv, self).__init__()

        # Load Pokémon data (each with type and moves)
        self.pokemon_data = self.load_pokemon_data(pokemon_data_path)

        # Type effectiveness chart (customize as needed)
        self.type_chart = {
            "Fire": {"Grass": 2.0, "Water": 0.5, "Fire": 0.5},
            "Water": {"Fire": 2.0, "Grass": 0.5, "Water": 0.5},
            "Grass": {"Water": 2.0, "Fire": 0.5, "Grass": 0.5},
            "Electric": {"Water": 2.0, "Grass": 0.5, "Electric": 0.5},
            "Normal": {},
            "Dark": {"Psychic": 2.0, "Fighting": 0.5},
            "Dragon": {"Dragon": 2.0}
            # Add other types as needed.
        }

        # Action space: 0-3 use a move; 4-7 switch to bench slot (0-indexed)
        self.action_space = Discrete(8)
        # Observation: [agent_active_hp, opponent_active_hp, last_agent_damage, last_opponent_damage, agent_remaining]
        self.observation_space = Box(
            low=np.array([0, 0, 0, 0, 1]),
            high=np.array([500, 500, 100, 100, 4]),
            dtype=np.float32
        )

        self.max_steps = 300
        # For self-play: store the opponent's Q-table (if provided)
        self.opponent_q_table = opponent_q_table
        self.reset()

    def load_pokemon_data(self, path):
        with open(path, 'r') as f:
            return json.load(f)

    def get_type_multiplier(self, move_type, target_type):
        return self.type_chart.get(move_type, {}).get(target_type, 1.0)

    def reset(self):
        self.steps = 0
        self.last_agent_damage = 0
        self.last_opponent_damage = 0

        # --- For the Agent ---
        # Select 4 random Pokémon for the agent.
        self.agent_bench = []
        agent_keys = random.sample(list(self.pokemon_data.keys()), 4)
        for key in agent_keys:
            pkmn = {
                "name": key,
                "type": self.pokemon_data[key]["type"],
                "moves": list(self.pokemon_data[key]["moves"].values()),
                "hp": 500
            }
            self.agent_bench.append(pkmn)
        # Choose one active Pokémon (randomly)
        self.agent_active_idx = random.randint(0, 3)

        # --- For the Opponent ---
        # Select 4 random Pokémon for the opponent.
        self.opponent_bench = []
        opponent_keys = random.sample(list(self.pokemon_data.keys()), 4)
        for key in opponent_keys:
            pkmn = {
                "name": key,
                "type": self.pokemon_data[key]["type"],
                "moves": list(self.pokemon_data[key]["moves"].values()),
                "hp": 500
            }
            self.opponent_bench.append(pkmn)
        # Choose one active Pokémon for the opponent (randomly)
        self.opponent_active_idx = random.randint(0, 3)

        return self._get_state()

    def _get_state(self):
        # Count how many agent Pokémon are still alive (hp > 0)
        agent_remaining = sum(1 for p in self.agent_bench if p["hp"] > 0)
        return np.array([
            self.agent_bench[self.agent_active_idx]["hp"],
            self.opponent_bench[self.opponent_active_idx]["hp"],
            self.last_agent_damage,
            self.last_opponent_damage,
            agent_remaining
        ], dtype=np.float32)

    def _get_opponent_state(self):
        # Get mirrored state from opponent's perspective
        opponent_remaining = sum(1 for p in self.opponent_bench if p["hp"] > 0)
        return np.array([
            self.opponent_bench[self.opponent_active_idx]["hp"],  # opponent's active HP
            self.agent_bench[self.agent_active_idx]["hp"],        # agent's active HP (opponent's enemy)
            self.last_opponent_damage,                           # last damage done by opponent
            self.last_agent_damage,                              # last damage done by agent (opponent's enemy)
            opponent_remaining                                    # opponent's remaining Pokémon
        ], dtype=np.float32)

    def _get_opponent_action(self, state):
        # If no Q-table is provided for the opponent, use random action
        if self.opponent_q_table is None:
            return self.action_space.sample()

        # Otherwise, use the opponent's Q-table for action selection
        state_idx = discretize_state(state)
        return np.argmax(self.opponent_q_table[state_idx])

    def step(self, action):
        HIT_POINTS = 500.0
        self.steps += 1
        reward = 0
        self.last_agent_damage = 0
        self.last_opponent_damage = 0

        # --- First, check if agent's active Pokémon is fainted and force a switch if needed ---
        if self.agent_bench[self.agent_active_idx]["hp"] <= 0:
            available = [i for i, p in enumerate(self.agent_bench) if p["hp"] > 0]
            if available:
                self.agent_active_idx = random.choice(available)
            else:
                # All Pokémon fainted - battle should end
                return self._get_state(), reward, True, {"info": "All agent Pokémon fainted"}

        # --- Agent Turn ---
        if action >= 4:  # Switching action
            new_idx = action - 4
            if self.agent_bench[new_idx]["hp"] <= 0:
                reward -= 0.5/HIT_POINTS
            elif new_idx == self.agent_active_idx:
                reward -= 0.2/HIT_POINTS  # Small penalty for wasted switch
            else:
                self.agent_active_idx = new_idx
                reward -= 0.05/HIT_POINTS  # Minor penalty for switching (since no damage is dealt)
            agent_damage = 0
        else:
            active_agent = self.agent_bench[self.agent_active_idx]
            # This check is redundant now since we checked earlier, but keeping for safety
            if active_agent["hp"] <= 0:
                agent_damage = 0
            else:
                move_idx = action
                if move_idx >= len(active_agent["moves"]):
                    move_idx = 0  # fallback if move index is out of range
                move = active_agent["moves"][move_idx]
                base_damage = move["damage"]
                multiplier = self.get_type_multiplier(move["type"],
                                                    self.opponent_bench[self.opponent_active_idx]["type"])
                agent_damage = base_damage * multiplier
                # Apply damage and clamp HP to a minimum of 0
                self.opponent_bench[self.opponent_active_idx]["hp"] = max(
                    0, self.opponent_bench[self.opponent_active_idx]["hp"] - agent_damage
                )
        # Reward is normalized damage dealt by the agent
        reward += agent_damage / HIT_POINTS
        self.last_agent_damage = agent_damage  # Store the damage for state observation

        # --- Check if opponent's active Pokémon is fainted and force a switch if needed ---
        if self.opponent_bench[self.opponent_active_idx]["hp"] <= 0:
            available = [i for i, p in enumerate(self.opponent_bench) if p["hp"] > 0]
            if available:
                self.opponent_active_idx = random.choice(available)
            else:
                # All opponent Pokémon fainted - battle should end
                return self._get_state(), reward + 1, True, {"info": "All opponent Pokémon fainted"}

        # --- Opponent Turn ---
        if self.opponent_q_table is not None:
            opponent_state = self._get_opponent_state()
            opponent_action = self._get_opponent_action(opponent_state)
        else:
            opponent_action = random.choice(range(self.action_space.n))

        if opponent_action >= 4:  # Opponent switching action
            new_idx = opponent_action - 4
            if new_idx != self.opponent_active_idx and self.opponent_bench[new_idx]["hp"] > 0:
                self.opponent_active_idx = new_idx
            opponent_damage = 0
        else:
            opp_active = self.opponent_bench[self.opponent_active_idx]
            # This check is redundant since we checked earlier, but keeping for safety
            if opp_active["hp"] <= 0:
                opponent_damage = 0
            else:
                move_idx = opponent_action
                if move_idx >= len(opp_active["moves"]):
                    move_idx = 0  # fallback if index out of range
                opp_move = opp_active["moves"][move_idx]
                opp_base_damage = opp_move["damage"]
                opp_multiplier = self.get_type_multiplier(opp_move["type"],
                                                        self.agent_bench[self.agent_active_idx]["type"])
                opponent_damage = opp_base_damage * opp_multiplier
                self.agent_bench[self.agent_active_idx]["hp"] = max(
                    0, self.agent_bench[self.agent_active_idx]["hp"] - opponent_damage
                )
        self.last_opponent_damage = opponent_damage  # Store the damage for state observation

        # --- Additional Reward for Knockouts ---
        # If the opponent's active Pokémon fainted after our attack, add a bonus.
        if self.opponent_bench[self.opponent_active_idx]["hp"] == 0:
            reward += 1

        # Determine if battle is over
        done = (all(p["hp"] <= 0 for p in self.agent_bench) or
                all(p["hp"] <= 0 for p in self.opponent_bench) or
                self.steps >= self.max_steps)

        return self._get_state(), reward, done, {}

    def render(self, mode='human'):
        print(f"Step: {self.steps}")
        print(f"Agent Active: {self.agent_bench[self.agent_active_idx]['name']} [{self.agent_bench[self.agent_active_idx]['hp']:.1f} HP]")
        print(f"Opponent Active: {self.opponent_bench[self.opponent_active_idx]['name']} [{self.opponent_bench[self.opponent_active_idx]['hp']:.1f} HP]")
        print("-" * 40)

# ===============================
# State Discretization Function (for Q-Learning)
# ===============================
def discretize_state(state):
    """
    State vector:
      [agent_active_hp, opponent_active_hp, last_agent_damage, last_opponent_damage, agent_remaining]

    We use predefined bins for HP and damage; agent_remaining is an integer (1 to 4).
    """
    # Define bins for HP and damage
    hp_bins = np.array([0, 100, 200, 300, 400, 500])
    damage_bins = np.array([0, 20, 40, 60, 80, 100])

    # Discretize agent active HP: cap maximum index at len(hp_bins)-3 so the max index is 3.
    hp1_idx = np.digitize(state[0], hp_bins) - 1
    hp1_idx = min(hp1_idx, len(hp_bins) - 3)

    # Discretize opponent active HP
    hp2_idx = np.digitize(state[1], hp_bins) - 1
    hp2_idx = min(hp2_idx, len(hp_bins) - 3)

    # Discretize last agent damage
    dmg1_idx = np.digitize(state[2], damage_bins) - 1
    dmg1_idx = min(dmg1_idx, len(damage_bins) - 3)

    # Discretize last opponent damage
    dmg2_idx = np.digitize(state[3], damage_bins) - 1
    dmg2_idx = min(dmg2_idx, len(damage_bins) - 3)

    # Agent remaining: values from 1 to 4 → subtract 1 to get an index from 0 to 3.
    remain_idx = int(state[4]) - 1
    remain_idx = min(max(remain_idx, 0), 3)

    return (hp1_idx, hp2_idx, dmg1_idx, dmg2_idx, remain_idx)


# ===============================
# Self-Play Q-Learning Training
# ===============================
hp_bins_count = 4       # valid indices 0 to 3
damage_bins_count = 4   # valid indices 0 to 3
remain_bins_count = 4   # for agent_remaining (1-4 mapped to 0-3)

num_actions = 8  # 4 moves + 4 switching actions
epsilon = 1.0
epsilon_min = 0.05
epsilon_decay = 0.9997
alpha = 0.1
gamma = 0.95

# Q-table shape: (hp1, hp2, dmg1, dmg2, remain, action)
q_table = np.zeros((hp_bins_count, hp_bins_count, damage_bins_count, damage_bins_count, remain_bins_count, num_actions))

# Self-play parameters
num_total_episodes = 20000
snapshot_interval = 5000  # Save a snapshot of the Q-table every 5000 episodes
q_table_snapshots = []
snapshot_record = []

env = EnhancedPokemonBattleEnv("/content/pokemon_trial.json")  # Initially no opponent Q-table
recent_rewards = []

# Main training loop with self-play
for episode in range(num_total_episodes):
    # Create a copy of the current Q-table for the opponent every snapshot_interval episodes
    if episode > 0 and episode % snapshot_interval == 0:
        snapshot = copy.deepcopy(q_table)
        q_table_snapshots.append(snapshot)
        snapshot_record.append(episode)

        # Update the environment to use the latest snapshot if available
        if len(q_table_snapshots) > 0:
            # Choose a random previous snapshot to play against (curriculum learning)
            opponent_q_idx = random.randint(0, len(q_table_snapshots) - 1)
            env = EnhancedPokemonBattleEnv(
                "/content/pokemon_trial.json",
                opponent_q_table=q_table_snapshots[opponent_q_idx]
            )
            print(f"Self-play started: Agent playing against snapshot from episode {snapshot_record[opponent_q_idx]}")

    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        state_idx = discretize_state(state)

        # Epsilon-greedy action selection
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state_idx])

        next_state, reward, done, _ = env.step(action)
        next_state_idx = discretize_state(next_state)

        # Q-learning update
        old_value = q_table[state_idx][action]
        next_max = np.max(q_table[next_state_idx])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state_idx][action] = new_value

        state = next_state
        total_reward += reward

    recent_rewards.append(total_reward)
    if len(recent_rewards) > 100:
        recent_rewards.pop(0)

    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    if (episode + 1) % 1000 == 0:
        avg_reward = np.mean(recent_rewards)
        print(f"Episode: {episode + 1}, Avg Reward: {avg_reward:.2f}, Epsilon: {epsilon:.3f}")

# ===============================
# Test the Agent Against Different Snapshots
# ===============================
def test_against_snapshot(agent_q_table, snapshot_q_table=None, num_episodes=10):
    test_env = EnhancedPokemonBattleEnv("/content/pokemon_trial.json", opponent_q_table=snapshot_q_table)
    wins = 0
    total_rewards = []

    for episode in range(num_episodes):
        state = test_env.reset()
        done = False
        total_reward = 0

        while not done:
            state_idx = discretize_state(state)
            action = np.argmax(agent_q_table[state_idx])
            state, reward, done, _ = test_env.step(action)
            total_reward += reward
            # Uncomment to see each step: test_env.render()

        if total_reward > 0:  # Simple win condition based on positive reward
            wins += 1
        total_rewards.append(total_reward)
        print(f"Test Episode {episode + 1} finished with total reward: {total_reward:.2f}")

    return wins, total_rewards

# Test final agent against random actions
print("\nTesting final agent against random opponent:")
random_wins, random_rewards = test_against_snapshot(q_table, None)
print(f"Win rate against random opponent: {(random_wins/10)*100:.2f}%")
print(f"Average reward: {np.mean(random_rewards):.2f}")

# Test final agent against each snapshot
for i, snapshot in enumerate(q_table_snapshots):
    print(f"\nTesting final agent against snapshot {i+1} (from episode {snapshot_record[i]}):")
    snapshot_wins, snapshot_rewards = test_against_snapshot(q_table, snapshot)
    print(f"Win rate against snapshot {i+1}: {(snapshot_wins/10)*100:.2f}%")
    print(f"Average reward: {np.mean(snapshot_rewards):.2f}")

# Final match: Test against the final agent itself
print("\nTesting final agent against itself:")
self_wins, self_rewards = test_against_snapshot(q_table, q_table)
print(f"Win rate against itself: {(self_wins/10)*100:.2f}%")
print(f"Average reward: {np.mean(self_rewards):.2f}")

/usr/local/lib/python3.11/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Episode: 1000, Avg Reward: 4.80, Epsilon: 0.741
Episode: 2000, Avg Reward: 5.08, Epsilon: 0.549
Episode: 3000, Avg Reward: 5.15, Epsilon: 0.407
Episode: 4000, Avg Reward: 5.21, Epsilon: 0.301
Episode: 5000, Avg Reward: 5.20, Epsilon: 0.223
Self-play started: Agent playing against snapshot from episode 5000
Episode: 6000, Avg Reward: 4.05, Epsilon: 0.165
Episode: 7000, Avg Reward: 4.44, Epsilon: 0.122
Episode: 8000, Avg Reward: 4.45, Epsilon: 0.091
Episode: 9000, Avg Reward: 4.51, Epsilon: 0.067
Episode: 10000, Avg Reward: 4.58, Epsilon: 0.050
Self-play started: Agent playing against snapshot from episode 10000
Episode: 11000, Avg Reward: 4.31, Epsilon: 0.050
Episode: 12000, Avg Reward: 4.36, Epsilon: 0.050
Episode: 13000, Avg Reward: 4.22, Epsilon: 0.050
Episode: 14000, Avg Reward: 4.25, Epsilon: 0.050
Episode: 15000, Avg Reward: 4.33, Epsilon: 0.050
Self-play started: Agent playing against snapshot from episode 10000
Episode: 16000, Avg Reward: 4.17, Epsilon: 0.050
Episode: 17000, Avg

In [6]:
def test_with_action_printing(agent_q_table, snapshot_q_table=None, num_episodes=1):
    test_env = EnhancedPokemonBattleEnv("/content/pokemon_trial.json", opponent_q_table=snapshot_q_table)

    for episode in range(num_episodes):
        state = test_env.reset()
        done = False
        total_reward = 0

        # Print initial team details for agent and opponent (only first 4 moves)
        print(f"\n=== Battle Episode {episode + 1} ===\n")
        print("Agent Team:")
        for idx, pkmn in enumerate(test_env.agent_bench):
            print(f"  [{idx+1}] {pkmn['name']} (HP: {pkmn['hp']:.1f}, Type: {pkmn['type']})")
            # Only display the first 4 moves
            for move_idx, move in enumerate(pkmn['moves'][:4]):
                move_name = move.get("name", f"Move {move_idx}")
                print(f"      - {move_name}: Damage {move['damage']}, Type: {move['type']}")
        print("\nOpponent Team:")
        for idx, pkmn in enumerate(test_env.opponent_bench):
            print(f"  [{idx+1}] {pkmn['name']} (HP: {pkmn['hp']:.1f}, Type: {pkmn['type']})")
            # Only display the first 4 moves
            for move_idx, move in enumerate(pkmn['moves'][:4]):
                move_name = move.get("name", f"Move {move_idx}")
                print(f"      - {move_name}: Damage {move['damage']}, Type: {move['type']}")
        print("\n========================================\n")

        # Start the battle loop
        while not done:
            # Get current active Pokémon for agent and opponent
            agent_active = test_env.agent_bench[test_env.agent_active_idx]
            opponent_active = test_env.opponent_bench[test_env.opponent_active_idx]
            print(f"Agent Active: {agent_active['name']} (HP: {agent_active['hp']:.1f})")
            print(f"Opponent Active: {opponent_active['name']} (HP: {opponent_active['hp']:.1f})")

            # Determine agent's action from Q-table
            state_idx = discretize_state(state)
            agent_action = np.argmax(agent_q_table[state_idx])

            # Describe agent's action in detail
            if agent_action < 4:
                # Attack action
                move_idx = agent_action
                # Fallback if the active Pokémon doesn't have enough moves
                if move_idx >= len(agent_active["moves"]):
                    move_idx = 0
                move = agent_active["moves"][move_idx]
                move_name = move.get("name", f"Move {move_idx}")
                print(f"Agent chooses to use move: {move_name} (Damage: {move['damage']}, Type: {move['type']})")
            else:
                # Switching action
                bench_idx = agent_action - 4
                target_pokemon = test_env.agent_bench[bench_idx]
                print(f"Agent chooses to switch to: {target_pokemon['name']} (HP: {target_pokemon['hp']:.1f})")

            # Determine opponent's action (if opponent's Q-table is provided)
            if test_env.opponent_q_table is not None:
                opponent_state = test_env._get_opponent_state()
                opp_action = test_env._get_opponent_action(opponent_state)
                if opp_action < 4:
                    opp_move_idx = opp_action
                    # Fallback if move index is out of range
                    if opp_move_idx >= len(opponent_active["moves"]):
                        opp_move_idx = 0
                    opp_move = opponent_active["moves"][opp_move_idx]
                    opp_move_name = opp_move.get("name", f"Move {opp_move_idx}")
                    opp_action_str = f"uses move: {opp_move_name} (Damage: {opp_move['damage']}, Type: {opp_move['type']})"
                else:
                    bench_idx = opp_action - 4
                    target_opp = test_env.opponent_bench[bench_idx]
                    opp_action_str = f"switches to: {target_opp['name']} (HP: {target_opp['hp']:.1f})"
            else:
                opp_action_str = "takes a random action (no Q-table provided)"
            print(f"Opponent will {opp_action_str}\n")

            # Execute the step (agent's action; opponent's action is computed inside step)
            next_state, reward, done, _ = test_env.step(agent_action)
            total_reward += reward

            # Print post-turn status
            agent_after = test_env.agent_bench[test_env.agent_active_idx]
            opponent_after = test_env.opponent_bench[test_env.opponent_active_idx]
            print(f"After turn: Agent's {agent_after['name']} HP: {agent_after['hp']:.1f} | Opponent's {opponent_after['name']} HP: {opponent_after['hp']:.1f}\n")

            state = next_state

        print(f"Battle finished. Total reward: {total_reward:.2f}\n")

        # Print final HP details for all Pokémon on both teams
        print("Final Agent Team Status:")
        for idx, pkmn in enumerate(test_env.agent_bench):
            print(f"  [{idx+1}] {pkmn['name']} - HP: {max(0, pkmn['hp']):.1f}")
        print("\nFinal Opponent Team Status:")
        for idx, pkmn in enumerate(test_env.opponent_bench):
            print(f"  [{idx+1}] {pkmn['name']} - HP: {max(0, pkmn['hp']):.1f}")
        print("\n========================================\n")
test_with_action_printing(q_table, snapshot_q_table=None, num_episodes=10)



=== Battle Episode 1 ===

Agent Team:
  [1] Chikorita (HP: 500.0, Type: Grass)
      - Move 0: Damage 40, Type: Normal
      - Move 1: Damage 60, Type: Rock
      - Move 2: Damage 55, Type: Grass
      - Move 3: Damage 45, Type: Grass
  [2] Psyduck (HP: 500.0, Type: Water)
      - Move 0: Damage 40, Type: Normal
      - Move 1: Damage 40, Type: Water
      - Move 2: Damage 50, Type: Psychic
      - Move 3: Damage 18, Type: Normal
  [3] Poliwag (HP: 500.0, Type: Water)
      - Move 0: Damage 40, Type: Water
      - Move 1: Damage 40, Type: Water
      - Move 2: Damage 15, Type: Normal
      - Move 3: Damage 85, Type: Normal
  [4] Sentret (HP: 500.0, Type: Normal)
      - Move 0: Damage 40, Type: Normal
      - Move 1: Damage 90, Type: Normal
      - Move 2: Damage 40, Type: Normal
      - Move 3: Damage 18, Type: Normal

Opponent Team:
  [1] Diglett (HP: 500.0, Type: Ground)
      - Move 0: Damage 40, Type: Normal
      - Move 1: Damage 30, Type: Ghost
      - Move 2: Damage 20, Type: 